In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

In [2]:
# Load data
X = np.load("X.npy")                    # Shape: (1500, 300, 12)
y = np.load("y.npy")                   # Shape: (1500, 500, 12)

# Predict next 1000 steps only
Y = y[:, 300:, :]                     # Shape: (1500, 200, 12)
print(X.shape, y.shape)
# Flatten time for MLP input and output
X_flat = X.reshape(1500, -1)           # (1000, 1000*12)
Y_flat = Y.reshape(1500, -1)           # (1000, 1000*12)

# Normalize to [-1, 1]
scaler_x = MinMaxScaler(feature_range=(-1, 1))
scaler_y = MinMaxScaler(feature_range=(-1, 1))

X_scaled = scaler_x.fit_transform(X_flat)
Y_scaled = scaler_y.fit_transform(Y_flat)

# Convert to tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
Y_tensor = torch.tensor(Y_scaled, dtype=torch.float32)

# Dataset and loader
full_dataset = TensorDataset(X_tensor, Y_tensor)

# Split into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

(1500, 300, 12) (1500, 500, 12)


In [3]:
# Define MLP
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.Tanh(),
            nn.Linear(2048, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, output_dim)
        )

    def forward(self, x):
        return self.net(x)


In [4]:
# Train function
def train(model, train_loader, val_loader, epochs=50, lr=1e-3):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            pred = model(x_batch)
            loss = criterion(pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                val_loss += criterion(model(x_val), y_val).item()
        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1} | Train Loss: {total_loss/len(train_loader):.6f} | Val Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "mlp_3body_best.pth")


In [ ]:
def evaluate_and_plot(model, dataset, sample_idx, scaler_y):
    import torch
    import matplotlib.pyplot as plt

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    # Get a sample
    x, y_true = dataset[sample_idx]  # x: (3600,), y_true: (2400,)
    x = x.unsqueeze(0).to(device)    # Add batch dimension → (1, 3600)

    with torch.no_grad():
        y_pred = model(x).cpu().squeeze(0).numpy()  # (2400,)

    # Reshape for inverse transform
    y_true = y_true.numpy().reshape(1, -1)  # (1, 2400)
    y_pred = y_pred.reshape(1, -1)          # (1, 2400)

    # Inverse transform to real coordinates
    y_true_real = scaler_y.inverse_transform(y_true).reshape(200, 12)
    y_pred_real = scaler_y.inverse_transform(y_pred).reshape(200, 12)

    # Plot trajectories
    colors = ['blue', 'green', 'orange']
    plt.figure(figsize=(8, 6))
    for body_idx in range(3):
        xi, yi = 4 * body_idx, 4 * body_idx + 1
        plt.plot(y_true_real[:, xi], y_true_real[:, yi], label=f'True Body {body_idx}', color="black")
        plt.plot(y_pred_real[:, xi], y_pred_real[:, yi], label=f'Predicted Body {body_idx}', color=colors[body_idx], linestyle='--')

    plt.title("Trajectory of All Bodies (x-y)")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()


In [ ]:
# Model setup
input_dim = X_tensor.shape[1]         # 1000 * 12
output_dim = Y_tensor.shape[1]        # 1000 * 12
model = MLP(input_dim, output_dim)

# Train model
train(model, train_loader, val_loader, epochs=50, lr=1e-3)

# Evaluate all predictions
model.eval()
with torch.no_grad():
    predictions = model(X_tensor).cpu().numpy()

# Inverse transform predictions
Y_pred_real = scaler_y.inverse_transform(predictions)
Y_true_real = Y_flat  # already unscaled

# Save predictions
np.save("predicted_y_next_1000.npy", Y_pred_real)


Epoch 1 | Train Loss: 0.048130 | Val Loss: 0.029503
Epoch 2 | Train Loss: 0.026089 | Val Loss: 0.026106
Epoch 3 | Train Loss: 0.023016 | Val Loss: 0.022619
Epoch 4 | Train Loss: 0.020613 | Val Loss: 0.021760
Epoch 5 | Train Loss: 0.020121 | Val Loss: 0.021694
Epoch 6 | Train Loss: 0.018514 | Val Loss: 0.020836
Epoch 7 | Train Loss: 0.017691 | Val Loss: 0.020700
Epoch 8 | Train Loss: 0.017462 | Val Loss: 0.021722
Epoch 9 | Train Loss: 0.016608 | Val Loss: 0.020917
Epoch 10 | Train Loss: 0.015686 | Val Loss: 0.019992
Epoch 11 | Train Loss: 0.014841 | Val Loss: 0.020062
Epoch 12 | Train Loss: 0.014366 | Val Loss: 0.020400
Epoch 13 | Train Loss: 0.013765 | Val Loss: 0.020924
Epoch 14 | Train Loss: 0.013238 | Val Loss: 0.020043
Epoch 15 | Train Loss: 0.012895 | Val Loss: 0.020560
Epoch 16 | Train Loss: 0.012387 | Val Loss: 0.021421
Epoch 17 | Train Loss: 0.012137 | Val Loss: 0.020162
Epoch 18 | Train Loss: 0.011204 | Val Loss: 0.020975
Epoch 19 | Train Loss: 0.010529 | Val Loss: 0.020522
Ep

: 

In [ ]:
evaluate_and_plot(model, full_dataset, sample_idx=2, scaler_y=scaler_y)